In [ ]:
from langchain_ollama import OllamaEmbeddings #  Generates embeddings (vector representations) for text using the Ollama model.
import faiss # Facebook AI Similarity Search library. It efficiently handles large vector search.
from langchain_community.vectorstores import FAISS  #  Wraps a FAISS index into a LangChain-friendly vectorstore.
from langchain_community.docstore.in_memory import InMemoryDocstore  #  Keeps your documents in memory for easy retrieval.

In [ ]:
embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")  
# model="nomic-embed-text" → the embedding model we’re using.

In [ ]:
!ollama pull nomic-embed-text  # downloads the model locally.
# Then run your embedding code
vector = embeddings.embed_query("Hello World")   # converts your query "Hello World" into a 1536-dimensional vector.
vector

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 970aa74c0a90: 100% ▕██████████████████▏ 274 MB                         
pulling c71d239df917: 100% ▕██████████████████▏  11 KB                         
pulling ce4a164fc046: 100% ▕██████████████████▏   17 B                         
pulling 31df23ea7daa: 100% ▕██████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


[-0.0068086036,
 -0.0013209877,
 -0.1713929,
 0.008481252,
 0.0057583125,
 0.06987866,
 -0.00016679369,
 -0.043082807,
 -0.014568477,
 -0.05409954,
 0.00048353738,
 0.039163508,
 0.027721988,
 0.08087763,
 0.04534548,
 -0.06293566,
 0.010287705,
 -0.02960825,
 -0.042814884,
 0.029634403,
 -0.003730822,
 -0.094318494,
 -0.00761835,
 0.038097665,
 0.092127316,
 -0.0143241715,
 -0.015047103,
 0.061606757,
 0.0065122233,
 -0.022007374,
 -0.0012141147,
 -0.010960399,
 -0.0002113811,
 0.015673473,
 0.039467745,
 0.0027335947,
 0.03259738,
 0.017250184,
 0.01628404,
 0.005894953,
 -0.004697547,
 -0.014825283,
 0.01210269,
 0.010205799,
 0.06594373,
 -0.0014970312,
 -0.0041841697,
 0.00031881288,
 0.086877465,
 -0.060592458,
 -0.018199582,
 0.0053434186,
 -0.0009156711,
 0.060235802,
 0.06724865,
 0.035397135,
 0.049658548,
 -0.061643075,
 0.02423905,
 0.034569584,
 0.021729533,
 0.04367034,
 0.032951094,
 0.06523748,
 -0.017518016,
 -0.033583615,
 -0.025228824,
 0.035459045,
 -0.0027383824,
 

In [ ]:
len(vector)  
index = faiss.IndexFlatL2(1536)  # a FAISS index for L2 distance (Euclidean distance) with vector dimension 1536.
index.ntotal , index.d  
"""
index.ntotal → how many vectors are in the index (0 for now).
index.d → the dimensionality of vectors (should be 1536).
"""

(0, 1536)

In [ ]:
vectorstore = FAISS(   #  Wraps your FAISS index + embeddings + docstore into a LangChain vectorstore.
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore({}),   #  holds the actual documents.
    index_to_docstore_id={},  # maps FAISS indices to document IDs.
)

In [6]:
db_name = "health_suppliments"
vectorstore = FAISS.load_local(db_name , embeddings , allow_dangerous_deserialization=True)

In [ ]:
# HOW QUERRY WORKS
question = "What are the benefits of taking vitamin C?"
docs = vectorstore.similarity_search(query = question , k=3)

"""
similarity_search → finds the top k=3 documents closest in meaning to your question.
Returns a list of matching documents from your docstore.
"""

In [ ]:
docs

"""
Simply prints the documents retrieved in the previous step.
Now you can see which documents FAISS thinks are most relevant.
"""

[Document(id='29eec152-9597-4a61-8ba3-7bc146f7eabf', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'rag-dataset\\regulatory_human_body-generic_manual.pdf', 'file_path': 'rag-dataset\\regulatory_human_body-generic_manual.pdf', 'total_pages': 17, 'format': 'PDF 1.4', 'title': 'manual_regulatory_document', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 4}, page_content='The kidneys also: \n●\u200b Control the acid-base balance (pH balance) of your blood. \n●\u200b Make sugar (glucose) if your blood doesn’t have enough sugar. \n●\u200b Make a protein called renin that increases blood pressure. \n●\u200b Produce the hormones calcitriol and erythropoietin. Calcitriol is a form of vitamin \nD that helps your body absorb calcium. Erythropoietin helps your body make red \nblood cells. \nAn adrenal gland sits on top of each kidney. It produces hormones, including cort

In [ ]:
retriver = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})  #  Converts your vectorstore into a LangChain retriever object.
retriver.invoke(question)   #  fetches the top k=3 docs just like similarity_search, but in a more LangChain-friendly format for pipelines like RAG.

[Document(id='29eec152-9597-4a61-8ba3-7bc146f7eabf', metadata={'producer': 'Skia/PDF m135 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'rag-dataset\\regulatory_human_body-generic_manual.pdf', 'file_path': 'rag-dataset\\regulatory_human_body-generic_manual.pdf', 'total_pages': 17, 'format': 'PDF 1.4', 'title': 'manual_regulatory_document', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 4}, page_content='The kidneys also: \n●\u200b Control the acid-base balance (pH balance) of your blood. \n●\u200b Make sugar (glucose) if your blood doesn’t have enough sugar. \n●\u200b Make a protein called renin that increases blood pressure. \n●\u200b Produce the hormones calcitriol and erythropoietin. Calcitriol is a form of vitamin \nD that helps your body absorb calcium. Erythropoietin helps your body make red \nblood cells. \nAn adrenal gland sits on top of each kidney. It produces hormones, including cort